In [3]:
import face_rec

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [4]:
import pandas as pd
import numpy as np
import cv2

In [5]:
face_rec.r.ping()

True

#### 1. Extract data from db

In [6]:
# face_rec.r.hdel('academy:register', 'Ahikmah@')
retrieve_dict = face_rec.r.hgetall('academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode('utf-8'), index))
retrieve_series.index = index

retrieve_series

Student@Chris Evans           [0.35691047, -0.41491055, 0.039218444, -0.1543...
Student@Scarlett Johansson    [-0.13535985, -0.2821385, -0.42700845, -0.2825...
Teacher@Barack Obama          [-0.22177827, 1.5762169, -0.5030781, 0.1360128...
Teacher@Morgan Freeman        [0.76278937, 0.890615, 0.46873552, 0.25898102,...
Student@Awaliyatul Hikmah     [-1.9841429, 0.21223877, 0.23982796, -0.489269...
Student@Angelina Jolie        [0.19881932, 1.4748837, -0.0979588, -0.8805054...
dtype: object

In [7]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['role_name', 'facial_features']
retrieve_df[['role', 'name']] = retrieve_df['role_name'].apply(lambda x: x.split('@')).apply(pd.Series)

retrieve_df

,role_name,facial_features,role,name
0,Student@Chris Evans,"[0.35691047, -0.41491055, 0.039218444, -0.1543...",Student,Chris Evans
1,Student@Scarlett Johansson,"[-0.13535985, -0.2821385, -0.42700845, -0.2825...",Student,Scarlett Johansson
2,Teacher@Barack Obama,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128...",Teacher,Barack Obama
3,Teacher@Morgan Freeman,"[0.76278937, 0.890615, 0.46873552, 0.25898102,...",Teacher,Morgan Freeman
4,Student@Awaliyatul Hikmah,"[-1.9841429, 0.21223877, 0.23982796, -0.489269...",Student,Awaliyatul Hikmah
5,Student@Angelina Jolie,"[0.19881932, 1.4748837, -0.0979588, -0.8805054...",Student,Angelina Jolie


#### 2. Get Realtime Prediction

In [8]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        # frame = cv2.flip(frame, 1)
        # frame = cv2.resize(frame, (640, 480))
        frame = face_rec.face_prediction(frame, retrieve_df, 'facial_features', thresh=0.5)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == 27: # press ESC to exit
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

d:\aloka-vision\venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\aloka-vision\venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\aloka-vision\venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max